## MGMTMSA 403 Homework 2: Portfolio Optimization


## Pre-processing step: Estimate expected returns and covariance

In [2]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickets of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## collect index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))


In [27]:
#Model 1
mod1 = Model()

#returns of the four stocks
r = [avg_return[x] for x in tickind]
s = mod1.addVars(4)

#get covaraiance for the relavent stocks
cov1 = np.zeros((4,4))
for x in range(4):
    for y in range(4):
        cov1[x][y] = C[tickind[x]][tickind[y]]

#Constriants
Int1_con = mod1.addConstr(sum(s[x] for x in range(4)) == 1)
return_con = mod1.addConstr(sum(s[x] * r[x] for x in range(4)) >= 0.005)
#Non Neg con
for x in range(4):
    mod1.addConstr(s[x] >= 0)

#Objective
mod1.setObjective(sum(sum(s[x] * s[y] * cov1[x][y] for x in range(4)) for y in range(4)), GRB.MINIMIZE)
mod1.update()
mod1.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x90a27fa8
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.87864836e+05 -1.87864836e+05  3.50e+03 1.91e-07  1.00e+06     0s
   1   5.03195868e+

In [28]:
mod1.objVal

0.00017749326516582024

In [5]:
#Model 2
mod2 = Model()

#returns of the four stocks
s = mod2.addVars(len(avg_return))

#get covaraiance for the relavent stocks
cov2 = np.zeros((len(avg_return),len(avg_return)))
for x in range(len(avg_return)):
    for y in range(len(avg_return)):
        cov2[x][y] = C[x][y]

#Constriants
Int1_con = mod2.addConstr(sum(s[x] for x in range(len(avg_return))) == 1)
return_con = mod2.addConstr(sum(s[x] * avg_return[x] for x in range(len(avg_return))) >= 0.005)
#Non Neg con
for x in range(len(avg_return)):
    mod2.addConstr(s[x] >= 0)

#Objective
mod2.setObjective(sum(sum(s[x] * s[y] * cov2[x][y] for x in range(len(avg_return))) for y in range(len(avg_return))), GRB.MINIMIZE)
mod2.update()
mod2.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0x004b4e8a
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.06s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.89821559e-13 -2.89821559e-13  3.90e+05 3.22e-07  1.00e+06     0s


In [6]:
mod2.objVal

2.8785756399295937e-05

In [48]:
#Model 3
mod3 = Model()

#returns of the four stocks
s = mod3.addVars(len(avg_return))
limits = mod3.addVars(len(avg_return), vtype = GRB.BINARY)
mod3.Params.MIPGap = 0.1

#get covaraiance for the relavent stocks
cov3 = np.zeros((len(avg_return),len(avg_return)))
for x in range(len(avg_return)):
    for y in range(len(avg_return)):
        cov3[x][y] = C[x][y]

#Constriants
Int1_con = mod3.addConstr(sum(s[x] for x in range(len(avg_return))) == 1)
return_con = mod3.addConstr(sum(s[x] * avg_return[x] for x in range(len(avg_return))) >= 0.005)

#Non Neg con
for x in range(len(avg_return)):
    mod3.addConstr(s[x] >= 0)

#limited transcation constraints
limit_con = mod3.addConstr(sum(limits[x] for x in range(len(avg_return))) <= 4)
for x in range(len(avg_return)):
    mod3.addConstr(s[x] * limits[x] == s[x])

#Objective
mod3.setObjective(sum(sum(s[x] * s[y] * cov3[x][y] for x in range(len(avg_return))) for y in range(len(avg_return))), GRB.MINIMIZE)
mod3.update()
mod3.optimize()

Changed value of parameter MIPGap to 0.1
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 393 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0x05bed5c5
Model has 76245 quadratic objective terms
Model has 390 quadratic constraints
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01

In [19]:
mod3.objVal

6.753470760728118e-05

# Questions

1.a) Model 1
<br> Optimal Risk: 0.00017749326516582024
<br> Solver Time: 0.09099960327148438
<br> 
<br> Weight: 
>MSFT: 23.7%
<br>         GS: 2.6%
<br>         PG: 0%
<br>         SCHP: 73.7%

1.b) Model 2
<br> Optimal Risk: 2.8785756399295937e-05
<br> Solver Time: 0.1480865478515625

1.c) Model 3
<br> Optimal Risk: 6.753470760728118e-05
<br> Solver Time: 43.41264343261719
<br> 
<br> Weight:
> CME: 12.64%
<br>LLY: 7.55%
<br>NVDA: 4.37%
<br>BND: 75.44%

2.a)
<br> Higher, because Model 2 has variable pool of 390, much larger than Model 1, thus more likely to get a better optimal solution
<br>
<br>2.b)
<br> Lower, because Model 3 has limited transcation, which limits its ability to make more optimal combionation of stocks

3.a)
<br> Optimal Time: 6.753470760728118e-05 
<br>remains the same.
<br>
<br> 3.b)
<br> Run Time: 27.106000900268555 
<br>lower